In [1]:
# importing libraries

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# for importing the passwords and other access information from the .env file

import os
from dotenv import load_dotenv
load_dotenv()

dbname = os.getenv('dbname')
host = os.getenv('host')
port = os.getenv('port')
user = os.getenv('user')
password = os.getenv('password')

In [3]:
# checking if it works

print(user)

yuliia


In [4]:
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.extract import connect_to_redshift

In [5]:
# connecting to redshift

connect = connect_to_redshift(dbname, host, port, user, password)

connection to redshift made


In [6]:
# checking the connection

query = """select *
           from bootcamp.online_transactions_cleaned ot
           limit 5
        """

online_transactions = pd.read_sql(query, connect)

In [7]:
# checking the connection

online_transactions.head()

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,536370,21791,VINTAGE HEADS AND TAILS CARD GAME,1.25,24,30.00,2010-12-01 08:45:00,u12583,France
1,536385,22783,SET 3 WICKER OVAL BASKETS W LIDS,19.95,1,19.95,2010-12-01 09:56:00,u1742,United Kingdom
2,536392,22128,PARTY CONES CANDY ASSORTED,1.25,12,15.00,2010-12-01 10:29:00,u13705,United Kingdom
3,536404,22469,HEART OF WICKER SMALL,1.65,12,19.80,2010-12-01 11:29:00,u16218,United Kingdom
4,536412,22274,FELTCRAFT DOLL EMILY,2.95,2,5.90,2010-12-01 11:49:00,u1792,United Kingdom


In the original dataset, "online_transactions_cleaned," negative quantities were present, indicating canceled orders. Canceled order invoices were identified by the prefix "C" followed by a six-digit number, while regular invoices comprised only six-digit numbers.

Simply removing all negative quantities was not a viable solution, as I would still have had erroneous original orders. This distinction was crucial because a substantial number of identical items appeared to have been ordered erroneously, and retaining them would have compromised the data's integrity.

The task of filtering out both canceled and original orders proved to be challenging due to the absence of a clear pattern for identifying orders that would be canceled at a later time.

Upon careful consideration, I realized that my segmentation analysis would involve working with aggregated data. Summarizing both negative and positive quantities, whether from original or canceled orders, would yield a net value of zero, which would not significantly impact my analysis.

Further investigation revealed that the dataset covered a time range from December 1, 2010, to December 9, 2011. Consequently, negative numbers persisted in canceled orders, as the original orders predated the dataset's start date.

As a result, I made the decision to exclude 43 customers from the total count of 4363 customers if their total order value was less than or equal to zero.

In [8]:
# fetching the dataset, removing customers that have negative numbers

query = """select *
           from bootcamp.online_transactions_cleaned
           where customer_id not in 
               (
                SELECT customer_id
                from bootcamp.online_transactions_cleaned otc 
                GROUP BY customer_id
                having sum(total_order_value) <= 0)
        """

online_transactions = pd.read_sql(query, connect)

In [9]:
online_transactions

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,536370,21791,VINTAGE HEADS AND TAILS CARD GAME,1.25,24,30.00,2010-12-01 08:45:00,u12583,France
1,536385,22783,SET 3 WICKER OVAL BASKETS W LIDS,19.95,1,19.95,2010-12-01 09:56:00,u1742,United Kingdom
2,536392,22128,PARTY CONES CANDY ASSORTED,1.25,12,15.00,2010-12-01 10:29:00,u13705,United Kingdom
3,536404,22469,HEART OF WICKER SMALL,1.65,12,19.80,2010-12-01 11:29:00,u16218,United Kingdom
4,536412,22274,FELTCRAFT DOLL EMILY,2.95,2,5.90,2010-12-01 11:49:00,u1792,United Kingdom
...,...,...,...,...,...,...,...,...,...
399317,581476,22998,TRAVEL CARD WALLET KEEP CALM,0.42,72,30.24,2011-12-09 08:48:00,u12433,Norway
399318,581478,23126,DOLLCRAFT GIRL AMELIE,4.95,4,19.80,2011-12-09 08:59:00,u17364,United Kingdom
399319,581493,79190B,RETRO PLASTIC POLKA TRAY,0.42,15,6.30,2011-12-09 10:10:00,u12423,Belgium
399320,581501,22807,SET OF 6 T-LIGHTS TOADSTOOLS,1.25,12,15.00,2011-12-09 10:46:00,u12985,United Kingdom


In [10]:
# saving dataset to a pkl. file

online_transactions.to_pickle('../data/data_prepared_for_analysis.pkl')